In [26]:
from bs4 import BeautifulSoup
import requests

In [27]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_sovereign_states")
html_content = response.content
soup = BeautifulSoup(html_content,"html.parser")

In [28]:
table_countries = soup.find("table")

In [29]:
countries_rows = table_countries.find("tbody").find_all("tr")

In [30]:
countries_rows[7].find_all("td")[1].text.replace("A ","").replace("\n","")

'UN member state'

In [31]:
countries_rows[7].find_all("td")[0].find("a")["href"]

'/wiki/Andorra'

In [32]:
country_list = list()
for country_row in countries_rows:
    try:
        columns = country_row.find_all("td")
        if columns[1].text.replace("A ","").replace("\n","") == 'UN member state':
            country_a = columns[0].find("a")
            country_element = {"title":country_a["title"],"link":country_a["href"]}
            country_list.append(country_element)
    except:
        pass

In [37]:
# country = country_list[0]
# response = requests.get("https://en.wikipedia.org" + country["link"])
# content = response.content
# soup_afgh = BeautifulSoup(content)

In [62]:
# div_content_element = soup_afgh.find("div",{"id":"mw-content-text"}).find(class_="mw-parser-output")

In [53]:
# start = False
# text = country["title"] + "\n"
# for element in div_content_element:
#     if start == True and element.name == "p":
#         text += element.text
#     elif element.name == "table":
#         start = True
#     elif element.name == "metadata":
#         break
    
        

In [56]:
# with open(country["title"]+".txt","w",encoding="utf8") as f:
#     f.write(text)

In [70]:
def get_country_main_info(country):
    response = requests.get("https://en.wikipedia.org" + country["link"])
    content = response.content
    soup = BeautifulSoup(content)
    start = False
    text = country["title"] + "\n"
    div_content_element = soup.find("div",{"id":"mw-content-text"}).find(class_="mw-parser-output")
    for element in div_content_element:
        if start == True and element.name == "p":
            text += element.text
        elif element.name == "table":
            start = True
        elif element.name == "metadata":
            break
    return text

In [71]:
file_text= ""
for country in country_list:
    country_text = get_country_main_info(country)
    if file_text == "":
        file_text = country_text
    else:
        file_text += "\n" + country_text

In [74]:
with open("All_countries_summary.txt","w",encoding="utf8") as f:
    f.write(file_text)

In [75]:
country_tables = list()

In [76]:
def get_country_table(country):
    response = requests.get("https://en.wikipedia.org" + country["link"])
    content = response.content
    soup = BeautifulSoup(content)
    table = soup.find("table",class_="infobox ib-country vcard")
    return table

In [77]:
for country in country_list:
    country_tables.append(get_country_table(country))

In [95]:
# output_html_content = ""
# for country_tab in country_tables:
#     output_html_content += str(country_tab).replace("//","https://")

In [96]:
# soup_out = BeautifulSoup(output_html_content,"html.parser")

In [97]:
# with open("countries_table.html","w",encoding="utf8") as f:
#     f.write(str(soup_out))

In [98]:
import pandas

In [271]:
df = pandas.DataFrame(columns=["Country","Capital","Demonym","Population","Area"])

In [104]:
# df.append( {"Country": "Latvia","Capital":"Riga","Demonym":"Latvian","Population":2000000,"Area":64000},ignore_index=True)

C:\Users\arturs.olekss\AppData\Local\Temp\ipykernel_1404\3774515319.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append( {"Country": "Latvia","Capital":"Riga","Demonym":"Latvian","Population":2000000,"Area":64000},ignore_index=True)


,Country,Capital,Demonym,Population,Area
0,Latvia,Riga,Latvian,2000000,64000


In [105]:
country_tab = country_tables[20] 

In [109]:
name = country_tab.find(class_="fn org country-name").text

In [113]:
rows = country_tab.find_all("tr")

In [172]:
for row in rows:
    try:
        head = row.find("th")
        if head.text == "Capital":
            capitals = row.find("td").find_all("a")
            capital_text = ""
            for capital in capitals:
                if capital_text == "":
                    capital_text = capital["title"]
                else:
                    capital_text += ";" + capital["title"]
        elif head.find("a")["title"] == "Demonym":
            denonym = row.find("td").find("a").text
    except:
        pass

In [162]:
capital_text

'La Paz;Sucre'

In [127]:
mrg_top_rows = country_tab.find_all(class_="mergedtoprow")

In [154]:
for mrg_top_row in mrg_top_rows:
    try:
        text = mrg_top_row.find("a").text.replace(" ","").replace("\n","")
        if text == "Area":
            area_row = mrg_top_row.next_sibling
        elif text == "Population":
            population_row = mrg_top_row.next_sibling
    except:
        pass

In [152]:
area = int(area_row.find("td").text.split("\xa0")[0].replace(",",""))

In [153]:
area

1098581

In [160]:
population = population_row.find("td").text.replace(",","").split("[")[0]

'12054379'

In [173]:
denonym

'Bolivian'

In [266]:
def get_country_details(country_tab):
    name = country_tab.find(class_="fn org country-name").text
    rows = country_tab.find_all("tr")
    for row in rows:
        try:
            head = row.find("th")
            if "Capital" in head.text:
                capitals = row.find("td").find_all("a")
                capital_text = ""
                for capital in capitals:
                    if capital_text == "":
                        capital_text = capital["title"]
                    else:
                        capital_text += ";" + capital["title"]
            elif head.find("a")["title"] == "Demonym":
                denonym = row.find("td").find("a").text
        except:
            pass
    
    mrg_top_rows = country_tab.find_all(class_="mergedtoprow")
    for mrg_top_row in mrg_top_rows:
        try:
            text = mrg_top_row.find("a").text.replace(" ","").replace("\n","")
            if text == "Area":
                area_row = mrg_top_row.next_sibling
            elif text == "Population":
                population_row = mrg_top_row.next_sibling
        except:
            pass
    area = float(area_row.find("td").text.split("\xa0")[0].split("[")[0].replace(",",""))
    population = population_row.find("td").text.replace(",","").split("[")[0]
    
    return {"Country": name,"Capital":capital_text,
             "Demonym":denonym,
            "Population":population,"Area":area}
    

In [276]:
for tab_country in country_tables:
    try:
        row = pandas.DataFrame(get_country_details(tab_country),index=[0])
        df = pandas.concat([df,row],ignore_index=True)
    except:
        pass

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [274]:
df

,Country,Capital,Demonym,Population,Area
